## एम्बेडिङहरू

हाम्रो अघिल्लो उदाहरणमा, हामीले `vocab_size` लम्बाइ भएका उच्च-आयामीय बाग-ऑफ-वर्ड्स भेक्टरहरूमा काम गरेका थियौं, र हामीले कम-आयामीय स्थानगत प्रतिनिधित्व भेक्टरहरूलाई स्पष्ट रूपमा sparse one-hot प्रतिनिधित्वमा रूपान्तरण गरिरहेका थियौं। यो one-hot प्रतिनिधित्व मेमोरी-कुशल छैन, साथै, प्रत्येक शब्दलाई एक-अर्काबाट स्वतन्त्र रूपमा व्यवहार गरिन्छ, अर्थात् one-hot एन्कोड गरिएको भेक्टरहरूले शब्दहरू बीचको कुनै पनि अर्थपूर्ण समानता व्यक्त गर्दैनन्।

यस इकाईमा, हामी **News AG** डेटासेटको अन्वेषण जारी राख्नेछौं। सुरु गर्नका लागि, डेटा लोड गरौं र अघिल्लो नोटबुकबाट केही परिभाषाहरू प्राप्त गरौं।


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## एम्बेडिङ भनेको के हो?

**एम्बेडिङ** को विचार भनेको शब्दहरूलाई कम-आयामीय घना भेक्टरहरूद्वारा प्रतिनिधित्व गर्नु हो, जसले कुनै प्रकारले शब्दको अर्थलाई झल्काउँछ। हामी पछि कसरी अर्थपूर्ण शब्द एम्बेडिङहरू निर्माण गर्ने भन्ने छलफल गर्नेछौं, तर अहिलेका लागि एम्बेडिङलाई शब्द भेक्टरको आयाम घटाउने तरिकाको रूपमा सोचौं। 

त्यसैले, एम्बेडिङ लेयरले शब्दलाई इनपुटको रूपमा लिन्छ, र निर्दिष्ट गरिएको `embedding_size` को आउटपुट भेक्टर उत्पादन गर्छ। यो अर्थमा, यो `Linear` लेयरसँग धेरै मिल्दोजुल्दो छ, तर यो एक-हट इन्कोड गरिएको भेक्टर लिने सट्टा, शब्दको नम्बरलाई इनपुटको रूपमा लिन सक्षम हुनेछ।

हाम्रो नेटवर्कमा पहिलो लेयरको रूपमा एम्बेडिङ लेयर प्रयोग गरेर, हामी ब्याग-अफ-वर्ड्स मोडेलबाट **एम्बेडिङ ब्याग** मोडेलमा स्विच गर्न सक्छौं, जहाँ हामी पहिलो पटक हाम्रो पाठका प्रत्येक शब्दलाई सम्बन्धित एम्बेडिङमा रूपान्तरण गर्छौं, र त्यसपछि ती सबै एम्बेडिङहरूमा `sum`, `average` वा `max` जस्ता कुनै एग्रिगेट फङ्सन गणना गर्छौं।  

![पाँच अनुक्रम शब्दहरूको लागि एम्बेडिङ क्लासिफायर देखाउने छवि।](../../../../../translated_images/ne/embedding-classifier-example.b77f021a7ee67eee.webp)

हाम्रो क्लासिफायर न्यूरल नेटवर्क एम्बेडिङ लेयरबाट सुरु हुनेछ, त्यसपछि एग्रिगेसन लेयर, र यसको माथि लीनियर क्लासिफायर हुनेछ:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### भिन्न-भिन्न आकारका भेरिएबल अनुक्रमसँग काम गर्ने

यो आर्किटेक्चरको परिणामस्वरूप, हाम्रो नेटवर्कका लागि मिनिब्याचहरू विशेष तरिकाले तयार गर्नुपर्ने हुन्छ। अघिल्लो युनिटमा, जब bag-of-words प्रयोग गरिँदै थियो, सबै BoW टेन्सरहरूको आकार मिनिब्याचमा `vocab_size` बराबर हुन्थ्यो, चाहे हाम्रो पाठ अनुक्रमको वास्तविक लम्बाइ जेसुकै किन नहोस्। जब हामी word embeddings तर्फ सर्छौं, प्रत्येक पाठ नमूनामा शब्दहरूको संख्या फरक-फरक हुन सक्छ, र ती नमूनाहरूलाई मिनिब्याचमा संयोजन गर्दा हामीले केही padding लागू गर्नुपर्ने हुन्छ।

यो काम datasource मा `collate_fn` function प्रदान गरेर गर्न सकिन्छ:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### एम्बेडिङ वर्गीकरणकर्ता प्रशिक्षण गर्दै

अब हामीले उपयुक्त डाटालोडर परिभाषित गरिसकेपछि, हामीले अघिल्लो इकाईमा परिभाषित गरेको प्रशिक्षण कार्य प्रयोग गरेर मोडेल प्रशिक्षण गर्न सक्छौं:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **नोट**: हामी यहाँ समय बचतको लागि मात्र २५ हजार रेकर्डहरू (एक पूर्ण युगभन्दा कम) को लागि प्रशिक्षण गर्दैछौं, तर तपाईं प्रशिक्षण जारी राख्न सक्नुहुन्छ, धेरै युगहरूको लागि प्रशिक्षण गर्नको लागि एक कार्य लेख्न सक्नुहुन्छ, र उच्च शुद्धता प्राप्त गर्नको लागि सिक्ने दर प्यारामिटरसँग प्रयोग गर्न सक्नुहुन्छ। तपाईं लगभग ९०% शुद्धतामा जान सक्षम हुनुहुनेछ।


### EmbeddingBag लेयर र भिन्न-लम्बाइ अनुक्रमको प्रतिनिधित्व

अघिल्लो आर्किटेक्चरमा, हामीले सबै अनुक्रमहरूलाई एउटै लम्बाइमा ल्याउनका लागि padding गर्नुपर्ने हुन्थ्यो ताकि तिनीहरूलाई मिनिब्याचमा फिट गर्न सकियोस्। यो भिन्न-लम्बाइ अनुक्रमहरूको प्रतिनिधित्व गर्ने सबैभन्दा प्रभावकारी तरिका होइन - अर्को तरिका भनेको **offset** भेक्टर प्रयोग गर्नु हो, जसले एउटै ठूलो भेक्टरमा भण्डारण गरिएका सबै अनुक्रमहरूको offsets राख्छ।

![Offset अनुक्रम प्रतिनिधित्व देखाउने चित्र](../../../../../translated_images/ne/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: माथिको चित्रमा, हामीले क्यारेक्टरहरूको अनुक्रम देखाएका छौं, तर हाम्रो उदाहरणमा हामी शब्दहरूको अनुक्रमसँग काम गरिरहेका छौं। यद्यपि, offset भेक्टर प्रयोग गरेर अनुक्रमहरूको प्रतिनिधित्व गर्ने सामान्य सिद्धान्त उस्तै रहन्छ।

Offset प्रतिनिधित्वसँग काम गर्नका लागि, हामी [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) लेयर प्रयोग गर्छौं। यो `Embedding` जस्तै छ, तर यसले सामग्री भेक्टर र offset भेक्टरलाई इनपुटको रूपमा लिन्छ, र यसमा औसत गर्ने लेयर पनि समावेश छ, जुन `mean`, `sum` वा `max` हुन सक्छ।

यहाँ `EmbeddingBag` प्रयोग गर्ने परिमार्जित नेटवर्क छ:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

डेटासेट प्रशिक्षणको लागि तयार गर्न, हामीले अफसेट भेक्टर तयार गर्ने रूपान्तरण कार्य प्रदान गर्न आवश्यक छ:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

ध्यान दिनुहोस्, कि सबै अघिल्लो उदाहरणहरू भन्दा फरक, हाम्रो नेटवर्कले अब दुई प्यारामिटरहरू स्वीकार गर्दछ: डेटा भेक्टर र अफसेट भेक्टर, जसको आकार फरक छ। त्यस्तै गरी, हाम्रो डेटा लोडरले पनि हामीलाई २ को सट्टा ३ मानहरू प्रदान गर्दछ: पाठ र अफसेट भेक्टरहरू दुवैलाई सुविधाहरूको रूपमा प्रदान गरिन्छ। त्यसैले, हामीले हाम्रो प्रशिक्षण कार्यलाई त्यसको हेरचाह गर्न अलिकति समायोजन गर्न आवश्यक छ:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## सेम्यान्टिक एम्बेडिङ: वर्ड२भेक

हाम्रो अघिल्लो उदाहरणमा, मोडेलको एम्बेडिङ तहले शब्दहरूलाई भेक्टर प्रतिनिधित्वमा म्याप गर्न सिक्यो, तर यो प्रतिनिधित्वमा धेरै सेम्यान्टिक अर्थ थिएन। यस्तो भेक्टर प्रतिनिधित्व सिक्न राम्रो हुने थियो, जहाँ समान शब्दहरू वा पर्यायवाची शब्दहरू केही भेक्टर दूरी (जस्तै, युक्लिडियन दूरी) को हिसाबले एक-अर्कासँग नजिक हुनेछन्।

यसका लागि, हामीले हाम्रो एम्बेडिङ मोडेललाई विशेष तरिकाले ठूलो पाठ संग्रहमा प्रि-ट्रेन गर्न आवश्यक छ। सेम्यान्टिक एम्बेडिङहरू ट्रेन गर्ने पहिलो तरिकाहरू मध्ये एकलाई [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) भनिन्छ। यो शब्दहरूको वितरित प्रतिनिधित्व उत्पादन गर्न प्रयोग गरिने दुई मुख्य आर्किटेक्चरहरूमा आधारित छ:

 - **कन्टिन्युअस ब्याग-अफ-वर्ड्स** (CBoW) — यस आर्किटेक्चरमा, हामी मोडेललाई वरपरको सन्दर्भबाट शब्दको भविष्यवाणी गर्न ट्रेन गर्छौं। दिइएको ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ मा, मोडेलको लक्ष्य $(W_{-2},W_{-1},W_1,W_2)$ बाट $W_0$ को भविष्यवाणी गर्नु हो।
 - **कन्टिन्युअस स्किप-ग्राम** CBoW को विपरीत हो। मोडेलले सन्दर्भ शब्दहरूको वरपरको विन्डो प्रयोग गरेर वर्तमान शब्दको भविष्यवाणी गर्छ।

CBoW छिटो हुन्छ, जबकि स्किप-ग्राम ढिलो हुन्छ, तर दुर्लभ शब्दहरूको प्रतिनिधित्व गर्न राम्रो काम गर्छ।

![CBoW र स्किप-ग्राम एल्गोरिदमहरू शब्दहरूलाई भेक्टरमा रूपान्तरण गर्ने तरिका देखाउने चित्र।](../../../../../translated_images/ne/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News डेटासेटमा प्रि-ट्रेन गरिएको word2vec एम्बेडिङसँग प्रयोग गर्न, हामी **gensim** लाइब्रेरी प्रयोग गर्न सक्छौं। तल हामी 'neural' शब्दसँग सबैभन्दा मिल्दो शब्दहरू फेला पार्छौं।

> **Note:** जब तपाईं पहिलो पटक शब्द भेक्टरहरू सिर्जना गर्नुहुन्छ, तिनीहरू डाउनलोड गर्न केही समय लाग्न सक्छ!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


हामी शब्दबाट भेक्टर एम्बेडिङहरू पनि गणना गर्न सक्छौं, जसलाई वर्गीकरण मोडेल प्रशिक्षणमा प्रयोग गर्न सकिन्छ (स्पष्टताको लागि हामी भेक्टरका पहिलो २० घटकहरू मात्र देखाउँछौं):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW र Skip-Grams दुबै "predictive" embeddings हुन्, किनकि यी केवल स्थानीय सन्दर्भहरूलाई मात्र ध्यानमा राख्छन्। Word2Vec ले विश्वव्यापी सन्दर्भको फाइदा उठाउँदैन।

**FastText** ले Word2Vec मा आधारित भएर प्रत्येक शब्द र शब्दभित्र पाइने अक्षर n-grams को लागि भेक्टर प्रतिनिधित्वहरू सिक्छ। प्रतिनिधित्वहरूको मानहरू प्रत्येक प्रशिक्षण चरणमा एक भेक्टरमा औसत गरिन्छ। यसले पूर्व-प्रशिक्षणमा धेरै अतिरिक्त गणना थपे पनि, यसले शब्द embeddings लाई उप-शब्द जानकारी समेट्न सक्षम बनाउँछ।

अर्को विधि, **GloVe**, सह-अवस्थित म्याट्रिक्सको विचारलाई उपयोग गर्दछ, सह-अवस्थित म्याट्रिक्सलाई अधिक अभिव्यक्तिपूर्ण र गैर-रेखीय शब्द भेक्टरहरूमा विघटन गर्न न्युरल विधिहरू प्रयोग गर्दछ।

तपाईं embeddings लाई FastText र GloVe मा परिवर्तन गरेर उदाहरणसँग खेल्न सक्नुहुन्छ, किनकि gensim ले विभिन्न शब्द embedding मोडेलहरूलाई समर्थन गर्दछ।


## PyTorch मा प्रि-ट्रेन गरिएको एम्बेडिङहरू प्रयोग गर्दै

हामी माथिको उदाहरणलाई संशोधन गरेर हाम्रो एम्बेडिङ लेयरको म्याट्रिक्सलाई Word2Vec जस्ता अर्थपूर्ण एम्बेडिङहरूद्वारा पहिले नै भरिएको बनाउन सक्छौं। हामीले ध्यान दिनुपर्छ कि प्रि-ट्रेन गरिएको एम्बेडिङको शब्दावली र हाम्रो पाठ कर्पसको शब्दावली सम्भवतः मेल खाने छैन, त्यसैले हामी हराएका शब्दहरूको लागि तौलहरूलाई र्यान्डम मानहरूद्वारा आरम्भ गर्नेछौं:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

हाम्रो अवस्थामा, हामीले शुद्धतामा ठूलो वृद्धि देख्दैनौं, जुन सम्भवतः धेरै फरक शब्दावलीका कारण हो।  
विभिन्न शब्दावलीको समस्यालाई समाधान गर्न, हामी निम्न समाधानहरू प्रयोग गर्न सक्छौं:  
* हाम्रो शब्दावलीमा आधारित word2vec मोडेल पुनः-प्रशिक्षण गर्नुहोस्  
* प्रि-ट्रेन गरिएको word2vec मोडेलको शब्दावली प्रयोग गरेर हाम्रो डेटासेट लोड गर्नुहोस्। डेटासेट लोड गर्दा प्रयोग गरिने शब्दावली निर्दिष्ट गर्न सकिन्छ।  

दोस्रो विधि सजिलो देखिन्छ, विशेष गरी किनभने PyTorch `torchtext` फ्रेमवर्कमा embeddings को लागि बिल्ट-इन समर्थन छ। उदाहरणका लागि, हामी GloVe-आधारित शब्दावली निम्न तरिकाले निर्माण गर्न सक्छौं:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


लोड गरिएको शब्दकोशमा निम्न आधारभूत कार्यहरू छन्:
* `vocab.stoi` शब्दकोशले हामीलाई शब्दलाई यसको शब्दकोश सूचकांकमा परिवर्तन गर्न अनुमति दिन्छ
* `vocab.itos` यसको उल्टो गर्छ - नम्बरलाई शब्दमा परिवर्तन गर्छ
* `vocab.vectors` एम्बेडिङ भेक्टरहरूको एरे हो, त्यसैले कुनै शब्द `s` को एम्बेडिङ प्राप्त गर्न हामीले `vocab.vectors[vocab.stoi[s]]` प्रयोग गर्नुपर्छ

यहाँ एम्बेडिङहरूलाई परिमार्जन गरेर समीकरण **kind-man+woman = queen** प्रदर्शन गर्ने उदाहरण छ (यसलाई काम गर्नका लागि मैले थोरै गुणांक समायोजन गर्नुपर्‍यो):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

हामीले ती एम्बेडिङहरू प्रयोग गरेर वर्गीकरणकर्ता प्रशिक्षण गर्न, पहिलो चरणमा हाम्रो डेटासेटलाई GloVe शब्दावली प्रयोग गरेर एन्कोड गर्न आवश्यक छ:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

जस्तो कि हामीले माथि देख्यौं, सबै भेक्टर एम्बेडिङहरू `vocab.vectors` म्याट्रिक्समा संग्रहित छन्। यसले ती वजनहरूलाई एम्बेडिङ तहको वजनहरूमा साधारण प्रतिलिपि गरेर लोड गर्न अत्यन्त सजिलो बनाउँछ:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

हाम्रो डाटासेटका केही शब्दहरू प्रि-ट्रेन गरिएको GloVe शब्दकोशमा नभएको कारणले गर्दा हामीले शुद्धतामा उल्लेखनीय वृद्धि देख्न नसकेको मुख्य कारणहरू मध्ये एक हो, र त्यसैले ती शब्दहरूलाई मूलतः बेवास्ता गरिएको छ। यस तथ्यलाई पार गर्नका लागि, हामी हाम्रो डाटासेटमा आफ्नै एम्बेडिङहरू प्रशिक्षण गर्न सक्छौं।


## सन्दर्भगत एम्बेडिङहरू

Word2Vec जस्ता परम्परागत प्रीट्रेन गरिएको एम्बेडिङ प्रतिनिधित्वहरूको एउटा मुख्य सीमाबद्धता भनेको शब्द अर्थ अस्पष्टताको समस्या हो। यद्यपि प्रीट्रेन गरिएको एम्बेडिङहरूले शब्दहरूको सन्दर्भमा केही अर्थ समेट्न सक्छन्, कुनै पनि शब्दको सबै सम्भावित अर्थ एउटै एम्बेडिङमा समेटिन्छ। यसले डाउनस्ट्रीम मोडेलहरूमा समस्या उत्पन्न गर्न सक्छ, किनभने धेरै शब्दहरू, जस्तै 'play' शब्द, प्रयोग गरिएको सन्दर्भ अनुसार फरक अर्थ राख्छन्।

उदाहरणका लागि, 'play' शब्दले यी दुई वाक्यहरूमा निकै फरक अर्थ राख्छ:
- म थिएटरमा एउटा **play** हेर्न गएँ।
- जोन आफ्ना साथीहरूसँग **play** गर्न चाहन्छ।

माथिका प्रीट्रेन गरिएको एम्बेडिङहरूले 'play' शब्दका यी दुवै अर्थहरू एउटै एम्बेडिङमा प्रतिनिधित्व गर्छन्। यस सीमाबद्धतालाई पार गर्नका लागि, हामीले **भाषा मोडेल**मा आधारित एम्बेडिङहरू निर्माण गर्न आवश्यक छ, जुन ठूलो पाठको संग्रहमा प्रशिक्षित गरिएको हुन्छ र *जान्दछ* कि शब्दहरू विभिन्न सन्दर्भहरूमा कसरी राख्न सकिन्छ। सन्दर्भगत एम्बेडिङहरूको चर्चा यस ट्युटोरियलको दायराभन्दा बाहिर पर्छ, तर हामी यसबारे भाषा मोडेलहरूको कुरा गर्दा अर्को युनिटमा फर्कनेछौं।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
